In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os

import numpy as np
import torch
import librosa
from torch.nn import functional as F
import matplotlib.pyplot as plt
import librosa.display
import IPython.display as ipd

from wavenet import model, train, sample, audio, datasets, utils, viz

In [ ]:
dry = False
if dry:
    os.environ['WANDB_MODE'] = 'dryrun'

In [ ]:
p = model.HParams(n_audio_chans=2)
ds = audio.load_dataset_from_track('data/aria.wav', p)
X, X_test = datasets.preprocess(ds, p)
X.shape

In [ ]:
m = model.Wavenet(p)
t = train.Trainer(m, X, X_test, train.HParams(max_epochs=100, batch_size=8, num_workers=8, learning_rate=0.0026), None)

In [ ]:
track_i = viz.plot_track(X, n_samples=p.receptive_field_size())

In [ ]:
ipd.Audio(audio.mu_expand(X[track_i][:p.receptive_field_size()], p), rate=p.sampling_rate)

In [ ]:
t.train()

In [ ]:
_, track = sample.sample(m, decoder=utils.decode_nucleus(), n_samples=32000, batch_size=10)

In [ ]:
viz.plot_track(track, n_samples=p.receptive_field_size() // 6)

In [ ]:
ipd.Audio(track[0], rate=p.sampling_rate)